In [ ]:
import urllib
import requests
from pandas.io.json import json_normalize
import json
import pandas as pd
import datetime
import time
import math
import numpy as np
from node import Node
from edge import Edge
from neural_network import NeuralNetwork

In [ ]:
url = 'https://rsbuddy.com/exchange/summary.json'
urljson = urllib.request.urlopen(url)
jsondata = json.loads(urljson.read().decode())

In [ ]:
#Get a list of all items
df = pd.read_json(path_or_buf='https://rsbuddy.com/exchange/summary.json',orient='index', convert_axes=True)
df = df[['id','name','buy_average','buy_quantity','sell_average','sell_quantity','overall_average','overall_quantity']]
data = df.sort_values(by=['id']).reset_index()
data = data.drop(labels='index',axis=1)

In [ ]:
#Run this to populate a csv: itemKey
itemKey = data[['id', 'name']]
itemKey.to_csv(path_or_buf='./itemKey.csv', columns=('id','name'), index=False)

In [ ]:
#Create test set of specified range number of items
data = pd.read_csv('./itemKey.csv', skiprows=[])
startNum = 0
numItems = 3
inputdata = data[startNum:startNum+numItems]
inputdata

In [ ]:
#Use this to pull a specific items by name
names = ['Armadyl godsword','Fire rune', 'Soft clay']

#read the itemKey csv and parse the key for associated item ids
data = pd.read_csv('./itemKey.csv', skiprows=[])
inputdata = pd.DataFrame()
for name in names:
    inputdata = inputdata.append(data.loc[data['name'] == name])
inputdata = inputdata.reset_index().drop(labels='index',axis=1)

print("Query List:")
print(inputdata)

In [ ]:
i = 0
firtIteration = True
urlquery = pd.DataFrame(columns=['id', 'name', 'data'])
#Iterate through all items, grab data from api and append to urlquery dataframe
while(i<inputdata['id'].count()):
    key = inputdata.iloc[i]['id']
    name = inputdata.iloc[i]['name']
    if (firtIteration == True):
        print("Item Id - " + str(key) + ": " + name)
    #Attempt to get data from API, retry if HTTP error
    try:
        url = f'https://api.rsbuddy.com/grandExchange?a=graph&g=240&start=1474615279000&i={key}'
        tempDataFrame = pd.DataFrame()
        tempDataFrame = pd.read_json(path_or_buf=url,orient='records', convert_axes=False)
        urlquery = urlquery.append({'id':key, 'name': name, 'data':tempDataFrame}, ignore_index=True)
        i+=1
        #check if there are more items to run. If no more items, return message indicating the process is done
        if (i == inputdata['id'].count()):
            print("It worked! That was the last item!")
        else:
            print("It worked! On to the next one.")
        firtIteration = True
    except:
        print("Got fucked. Trying again. :)")
        time.sleep(1)
        firtIteration = False
#Reformat urlquery columns, convert timestamp from miliseconds to seconds, convert timestamp to date, and sort by date
for index, row in urlquery.iterrows():
    row['data'] = row['data'][['ts','buyingPrice','buyingCompleted','sellingPrice','sellingCompleted','overallPrice','overallCompleted']]
    row['data'] = row['data'].sort_values(by=['ts'],ascending=1).reset_index()
    count = 0 
    for ind, r in row['data'].iterrows():
        r['ts'] = r['ts']/1000
        row['data'].loc[ind, 'ts'] = int(r['ts'])
        row['data'].loc[ind, 'date'] = datetime.datetime.fromtimestamp(r['ts']).isoformat()
    row['data'] = row['data'].drop(labels='index',axis=1)

In [ ]:
test_set_size = 360 #In Days - define the number of epochs to include in the test data

#create test_item object to log training and test sets
test_item = pd.DataFrame(columns=['id', 'name', 'train_x', 'train_y', 'test_x', 'test_y', 'pred_y'])
h = 0
#iterate through the urlquery results to generate training sets
while(h<urlquery['id'].count()):  
    item_id = urlquery.iloc[h]['id'] #Define the item to be queried for
    print("Item ID: " + str(item_id))
    #takes the item_id and generates the test data for the specified parameters as an array
    test_set = urlquery.loc[urlquery['id'] == item_id]
    train_x_headers = list(urlquery.iloc[0]['data'].columns.values[0:5])
    
    #for column key, uncomment this line below:
    #print(train_x_headers)

    #create training datasets
    train_x = test_set.iloc[0]['data'].iloc[0:math.ceil(test_set_size*6)].values[:,0:5]
    train_y = test_set.iloc[0]['data'].iloc[0:math.ceil(test_set_size*6)].values[:,5]
    print ("The shape of the " + test_set.iloc[0]['name'] +  " input array is: "+ str(train_x.shape))

    #create test set to estimate next 6 prices (next day of prices)
    test_x = test_set.iloc[0]['data'].iloc[math.ceil(test_set_size*6):math.ceil(test_set_size*6)+6].values[:,0:5]

    #test key to compare to Y-hat
    test_y = test_set.iloc[0]['data'].iloc[math.ceil(test_set_size*6):math.ceil(test_set_size*6)+6].values[:,5]
    test_item = test_item.append({'id':item_id, 'name': test_set.iloc[0]['name'], 'train_x':train_x, 'train_y':train_y, 'test_x':test_x, 'test_y': test_y}, ignore_index=True)
    print("Record completed at test_item position [" + str(h) + "]")
    h += 1
    #TODO: Get this all in a for loop to iterate over multiple entries

In [ ]:
#Test the creation of a neural network
nn = NeuralNetwork()
nn.generateInitialNetwork([1,1,1])
nn.calculateNodeActivations()
print(nn.outputNode.getActivation())



<font size="+3"><b>Testing of network size:</b></font>

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
#import keras.backend as K
train_x = test_item.iloc[2]['train_x'] # test data. not real
 
def create_model(layesr,activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=train_x.shape[1]))
            model.add(Activation(activation))
        else: 
            model.add(Dense(nodes))
            model.add(Activation(activation))
    model.add(Dense(1)) #Note: no activations present beyond this point

    model.compile(optimizer='adadelta', loss='mse')
    return model

model = KerasRegressor(build_fn=create_model, verbose = 0)

In [ ]:
layers = [[16], [4,2], [4], [16,4]]
activations = [tanh, relu]
param_grid = dict(layers=layers, activation=activations, batch_size = [42, 180], epochs[6])
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error')

In [27]:
import urllib
import requests
from pandas.io.json import json_normalize
import json
import pandas as pd
import datetime
import time
import math
import numpy as np

from neural_network import NeuralNetwork

"""
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.layers import Activation, Dense
from keras.models import Sequential
"""


def generate_item_records_from_summary():
    """ Generate item records using the summary.json link from the OSBuddy API """

    df = pd.read_json(path_or_buf='https://rsbuddy.com/exchange/summary.json',orient='index', convert_axes=True)
    df = df[['id','name','buy_average','buy_quantity','sell_average','sell_quantity','overall_average','overall_quantity']]
    data = df.sort_values(by=['id']).reset_index()
    data = data.drop(labels='index',axis=1)

    #Output item id/name pairs to a csv file
    item_key = data[['id', 'name']]
    file_name = './item_key.csv'
    item_key.to_csv(path_or_buf=file_name, columns=('id','name'), index=False)


def generate_input_data_by_item_number(start_num, num_items): 
    """Pull items by range of Index values"""
    file_name = './item_key.csv'
    all_items = pd.read_csv(file_name, skiprows=[])
    items = all_items[start_num: start_num + num_items]
    print("Item List generated by item number:")
    print(items)
    return items

def generate_input_data_by_item_name(names):  #Pull a specific items by name
    """Pull specific items by name"""
    file_name = './item_key.csv'
    data = pd.read_csv(file_name, skiprows=[])
    items = pd.DataFrame()
    for name in names:
        items = items.append(data.loc[data['name'] == name])
    items = items.reset_index().drop(labels='index',axis=1)
    print("Item List generated by name:")
    print(items)
    return items

def get_item_records_from_url(input_data, test_set_size):
    """ Iterate through all items, grab data from api and append to dataframe """
    i = 0
    item_lookup_failed = False
    item_records = pd.DataFrame(columns=['id', 'name', 'data'])
    while(i<input_data['id'].count()):
        key = input_data.iloc[i]['id']
        name = input_data.iloc[i]['name']
        #Print item information only on first data retrieval attempt 
        if not item_lookup_failed:
            print('Querying API for ' + name + ' data. ' + 'Item Id: ' + str(key))
        try:
            #Attempt to get data from API, retry if HTTP error
            url = f'https://api.rsbuddy.com/grandExchange?a=graph&g=240&start=1474615279000&i={key}'
            temp_df = pd.DataFrame()
            temp_df = pd.read_json(path_or_buf=url,orient='records', convert_axes=False)
            item_records = item_records.append({'id':key, 'name': name, 'data':temp_df}, ignore_index=True)
            i+=1
            item_lookup_failed = False
        except:
            print("Retrying...")
            time.sleep(1) #Avoid getting blacklisted by API
            item_lookup_failed = True
    print('Item retrieval complete!')

    #Add correct formatting to item record dates/times
    item_records = format_item_record_dates(item_records)

    return item_records

def format_item_record_dates(item_records):
    """ Converts timestamp to Unix seconds, and tacks on formatted date field """

    print("Formatting item record dates...")
    for _, row in item_records.iterrows():
        row['data'] = row['data'][['ts','buyingPrice','buyingCompleted','sellingPrice','sellingCompleted', 'overallPrice','overallCompleted']]
        row['data'] = row['data'].sort_values(by=['ts'],ascending=1).reset_index()
        for ind, r in row['data'].iterrows():
            r['ts'] = int(r['ts']/1000)
            row['data'].loc[ind, 'ts'] = int(r['ts'])
        timestamps = pd.DataFrame(columns=['ts','tsFromCurrent', 'tsYtd'],data=row['data']['ts'])
        print(timestamps)
        lastTs = timestamps['ts'][len(timestamps)-1]
        i = 0
        print ('Finding timestamps from most recent datapoint.')
        while(i<len(timestamps)):
            timestamps['tsFromCurrent'][i] = abs(timestamps['ts'][i] - lastTs)    
            i += 1
        print(timestamps['tsFromCurrent'])
        years = pd.DataFrame(columns=['year', 'unix'])    
        i = 0
        for year in range(2015,2050):
            unix = (year - 1970) * 31557600
            years.loc[i] = [year,unix]
            i+=1

        i = 0
        j = 0
        newYear = True
        while(i<len(timestamps)):
            unixYear = years.iloc[j]['unix']
            if newYear == True:
                print("Checking entries for year: " + str(years.iloc[j]['year']))
            ytd = abs(timestamps['ts'][i] - unixYear)
            if(ytd < 31557600):
                ytd = abs(timestamps.iloc[i]['ts'] - unixYear)
                timestamps['tsYtd'][i] = int(ytd)
                i += 1
                newYear = False
            else: 
                print("Not in this year. Going to next year...")
                j += 1
                newYear = True
        for ind, r in row['data'].iterrows():
            row['data'].loc[ind, 'date'] = datetime.datetime.fromtimestamp(r['ts']).isoformat()
            row['data'].loc[ind, 'tsFromCurrent'] = timestamps.iloc[ind]['tsFromCurrent']
            row['data'].loc[ind, 'tsYtd'] = timestamps.iloc[ind]['tsYtd']            
        row['data'] = row['data'].drop(labels='index',axis=1)
    return item_records

def generate_train_and_test_deltas(item_records):
    """ 
        Iterate through each item record and generate percentage deltas for buy price, 
        buy completed, sell price, sell completed, overall price, and overall completed
        compared to last sample period. 
    """
    for _, row in item_records.iterrows():
        for col_name in ['buyingPricePer', 'buyingCompletedPer', 'sellingPricePer', 
                        'sellingCompletedPer', 'overallPricePer', 'overallCompletedPer']:
            series_length = len(row['data']['ts'])
            row['data'][col_name] = pd.Series(np.random.randn(series_length), index=row['data'].index)
        #Each row represents and item, which has an associated dataframe 
        for index, _ in row['data'].iterrows():
            #Use data from last row to calc percentage change
            for col_name in ['buyingPrice', 'buyingCompleted', 'sellingPrice', 'sellingCompleted', 
                            'overallPrice', 'overallCompleted']:
                if(index == 0):
                    row['data'].loc[index, col_name + 'Per'] = 0
                else:
                    row['data'].loc[index, col_name + 'Per'] = calculate_percentage_change(row['data'], col_name, index)
                
                
            
def calculate_percentage_change(dataframe, columnName, index):
    """Calculate delta from last time stamp"""
    return (dataframe.loc[index, columnName] - dataframe.loc[index-1, columnName])/dataframe.loc[index-1, columnName]


def createTrainAndTestSet(item_records):
    print("Creating Test and Training Datasets...")
    #create test_item object to log training and test sets
    test_item = pd.DataFrame(columns=['id', 'name', 'train_val', 'test_val', 'train_per', 'test_per', 'pred_per'])
    record_index = 0
    #iterate through the urlquery results to generate training sets
    while(record_index < item_records['id'].count()):  
        item_id = item_records.iloc[record_index]['id'] 
        print("Creating Datasets for item ID: " + str(item_id))
        
        #takes the item_id and generates the test data for the specified parameters as an array
        test_set = item_records.loc[item_records['id'] == item_id]
        
        #for column key, uncomment this line below:
        #train_x_headers = list(urlquery.iloc[0]['data'].columns.values[0:6])
        #print(train_x_headers)

        #create training datasets
        train_val = test_set.iloc[0]['data'].iloc[0:math.ceil(test_set_size*6)].values[:,0:6]
        print ("The shape of the " + test_set.iloc[0]['name'] +  " input array is: "+ str(train_val.shape))

        #create test set to estimate next 6 prices (next day of prices)
        test_val = test_set.iloc[0]['data'].iloc[math.ceil(test_set_size*6):math.ceil(test_set_size*6)+6].values[:,0:6]

        #Generate deltas
        generate_train_and_test_deltas(item_records)

        #test key to compare to Y-hat
        test_item = 0
        #test_item = test_item.append({'id':item_id, 'name': test_set.iloc[0]['name'], 'train_val': train_val, 'test_val': test_val, 'train_per': train_per, 'test_per': test_per}, ignore_index=True)
        print("Record completed at test_item position [" + str(record_index) + "]")
        record_index += 1

    return test_item

if __name__ == '__main__':
    #Get a list of all items
    generate_item_records_from_summary()

    #Pull items by range of Index values
    input_data = generate_input_data_by_item_number(0,3)
    
    #Pull specific items by name
    #input_data = generate_input_data_by_item_name(['Armadyl godsword','Fire rune', 'Soft clay'])

    #In Days - define the number of epochs to include in the test data
    test_set_size = 30
    item_records = get_item_records_from_url(input_data, test_set_size)

    #Create test_item object to log training and test sets
    test_item = createTrainAndTestSet(item_records)

    #Test creation of Neural Net
    nn = NeuralNetwork()
    nn.generateInitialNetwork([1,1])
    nn.calculateNodeActivations()
    print(nn.outputNode.getActivation())

    #TODO: Create Year time stamp for seasonality
    #TODO: Create time from today time stamp
    #TODO: Create training and test sets
    #TODO: Test model size


    #random shit
    #import keras.backend as K
    """
    train_val = test_item.iloc[2]['train_val'] # test data. not real
    train_y = test_item.iloc[2]['train_y'] # test data. not real

    
    def create_model(layers,activation):
        model = Sequential()
        for i, nodes in enumerate(layers):
            if i==0:
                model.add(Dense(nodes,input_dim=train_val.shape[1]))
                model.add(Activation(activation))
            else: 
                model.add(Dense(nodes))
                model.add(Activation(activation))
        model.add(Dense(1)) #Note: no activations present beyond this point

        model.compile(optimizer='adadelta', loss='mse')
        return model

    print('creating model')
    model = KerasRegressor(build_fn=create_model, verbose = 0)
    layers = [[16], [4,2], [4], [16,4]]
    activations = ['tanh', 'relu']
    param_grid = dict(layers=layers, activation=activations, batch_size = [42, 180], epochs=[6])
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error')

    # grid_result = grid.fit(train_x, train_y) testing for network size """

Item List generated by item number:
   id          name
0   2    Cannonball
1   6   Cannon base
2   8  Cannon stand
Querying API for Cannonball data. Item Id: 2
Retrying...
Retrying...
Retrying...
Querying API for Cannon base data. Item Id: 6
Retrying...
Retrying...
Retrying...
Retrying...
Retrying...
Retrying...
Retrying...
Retrying...
Retrying...
Querying API for Cannon stand data. Item Id: 8
Retrying...
Item retrieval complete!
Formatting item record dates...
              ts tsFromCurrent tsYtd
0     1474603200           NaN   NaN
1     1474617600           NaN   NaN
2     1474632000           NaN   NaN
3     1474646400           NaN   NaN
4     1474660800           NaN   NaN
5     1474675200           NaN   NaN
6     1474689600           NaN   NaN
7     1474704000           NaN   NaN
8     1474718400           NaN   NaN
9     1474732800           NaN   NaN
10    1474747200           NaN   NaN
11    1474761600           NaN   NaN
12    1474776000           NaN   NaN
13    147479040

C:\Users\Jonathan Stav\Anaconda3\lib\site-packages\ipykernel_launcher.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0       59659200
1       59644800
2       59630400
3       59616000
4       59601600
5       59587200
6       59572800
7       59558400
8       59544000
9       59529600
10      59515200
11      59500800
12      59486400
13      59472000
14      59457600
15      59443200
16      59428800
17      59414400
18      59400000
19      59385600
20      59371200
21      59356800
22      59342400
23      59328000
24      59313600
25      59299200
26      59284800
27      59270400
28      59256000
29      59241600
          ...   
3978      417600
3979      403200
3980      388800
3981      374400
3982      360000
3983      345600
3984      331200
3985      316800
3986      302400
3987      288000
3988      273600
3989      259200
3990      244800
3991      230400
3992      216000
3993      201600
3994      187200
3995      172800
3996      158400
3997      144000
3998      129600
3999      115200
4000      100800
4001       86400
4002       72000
4003       57600
4004       43200
4005       288

C:\Users\Jonathan Stav\Anaconda3\lib\site-packages\ipykernel_launcher.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Not in this year. Going to next year...
unixYear is 1451649600
Checking entries for year: 2016
ytd is 22953600
unixYear is 1451649600
ytd is 22968000
unixYear is 1451649600
ytd is 22982400
unixYear is 1451649600
ytd is 22996800
unixYear is 1451649600
ytd is 23011200
unixYear is 1451649600
ytd is 23025600
unixYear is 1451649600
ytd is 23040000
unixYear is 1451649600
ytd is 23054400
unixYear is 1451649600
ytd is 23068800
unixYear is 1451649600
ytd is 23083200
unixYear is 1451649600
ytd is 23097600
unixYear is 1451649600
ytd is 23112000
unixYear is 1451649600
ytd is 23126400
unixYear is 1451649600
ytd is 23140800
unixYear is 1451649600
ytd is 23155200
unixYear is 1451649600
ytd is 23169600
unixYear is 1451649600
ytd is 23184000
unixYear is 1451649600
ytd is 23198400
unixYear is 1451649600
ytd is 23212800
unixYear is 1451649600
ytd is 23227200
unixYear is 1451649600
ytd is 23241600
unixYear is 1451649600
ytd is 23256000
unixYear is 1451649600
ytd is 23270400
unixYear is 1451649600
ytd is 

...

ytd is 27424800
unixYear is 1483207200
ytd is 27439200
unixYear is 1483207200
ytd is 27453600
unixYear is 1483207200
ytd is 27468000
unixYear is 1483207200
ytd is 27482400
unixYear is 1483207200
ytd is 27496800
unixYear is 1483207200
ytd is 27511200
unixYear is 1483207200
ytd is 27525600
unixYear is 1483207200
ytd is 27540000
unixYear is 1483207200
ytd is 27554400
unixYear is 1483207200
ytd is 27568800
unixYear is 1483207200
ytd is 27583200
unixYear is 1483207200
ytd is 27597600
unixYear is 1483207200
ytd is 27612000
unixYear is 1483207200
ytd is 27626400
unixYear is 1483207200
ytd is 27640800
unixYear is 1483207200
ytd is 27655200
unixYear is 1483207200
ytd is 27669600
unixYear is 1483207200
ytd is 27684000
unixYear is 1483207200
ytd is 27698400
unixYear is 1483207200
ytd is 27712800
unixYear is 1483207200
ytd is 27727200
unixYear is 1483207200
ytd is 27741600
unixYear is 1483207200
ytd is 27756000
unixYear is 1483207200
ytd is 27770400
unixYear is 1483207200
ytd is 27784800
unixYear 

unixYear is 1514764800
ytd is 4968000
unixYear is 1514764800
ytd is 4982400
unixYear is 1514764800
ytd is 4996800
unixYear is 1514764800
ytd is 5011200
unixYear is 1514764800
ytd is 5025600
unixYear is 1514764800
ytd is 5040000
unixYear is 1514764800
ytd is 5054400
unixYear is 1514764800
ytd is 5068800
unixYear is 1514764800
ytd is 5083200
unixYear is 1514764800
ytd is 5097600
unixYear is 1514764800
ytd is 5112000
unixYear is 1514764800
ytd is 5126400
unixYear is 1514764800
ytd is 5140800
unixYear is 1514764800
ytd is 5155200
unixYear is 1514764800
ytd is 5169600
unixYear is 1514764800
ytd is 5184000
unixYear is 1514764800
ytd is 5198400
unixYear is 1514764800
ytd is 5212800
unixYear is 1514764800
ytd is 5227200
unixYear is 1514764800
ytd is 5241600
unixYear is 1514764800
ytd is 5256000
unixYear is 1514764800
ytd is 5270400
unixYear is 1514764800
ytd is 5284800
unixYear is 1514764800
ytd is 5299200
unixYear is 1514764800
ytd is 5313600
unixYear is 1514764800
ytd is 5328000
unixYear is 

unixYear is 1514764800
ytd is 10296000
unixYear is 1514764800
ytd is 10310400
unixYear is 1514764800
ytd is 10324800
unixYear is 1514764800
ytd is 10339200
unixYear is 1514764800
ytd is 10353600
unixYear is 1514764800
ytd is 10368000
unixYear is 1514764800
ytd is 10382400
unixYear is 1514764800
ytd is 10396800
unixYear is 1514764800
ytd is 10411200
unixYear is 1514764800
ytd is 10425600
unixYear is 1514764800
ytd is 10440000
unixYear is 1514764800
ytd is 10454400
unixYear is 1514764800
ytd is 10468800
unixYear is 1514764800
ytd is 10483200
unixYear is 1514764800
ytd is 10497600
unixYear is 1514764800
ytd is 10512000
unixYear is 1514764800
ytd is 10526400
unixYear is 1514764800
ytd is 10540800
unixYear is 1514764800
ytd is 10555200
unixYear is 1514764800
ytd is 10569600
unixYear is 1514764800
ytd is 10584000
unixYear is 1514764800
ytd is 10598400
unixYear is 1514764800
ytd is 10612800
unixYear is 1514764800
ytd is 10627200
unixYear is 1514764800
ytd is 10641600
unixYear is 1514764800
yt

unixYear is 1514764800
ytd is 14040000
unixYear is 1514764800
ytd is 14054400
unixYear is 1514764800
ytd is 14068800
unixYear is 1514764800
ytd is 14083200
unixYear is 1514764800
ytd is 14097600
unixYear is 1514764800
ytd is 14112000
unixYear is 1514764800
ytd is 14126400
unixYear is 1514764800
ytd is 14140800
unixYear is 1514764800
ytd is 14155200
unixYear is 1514764800
ytd is 14169600
unixYear is 1514764800
ytd is 14184000
unixYear is 1514764800
ytd is 14198400
unixYear is 1514764800
ytd is 14212800
unixYear is 1514764800
ytd is 14227200
unixYear is 1514764800
ytd is 14241600
unixYear is 1514764800
ytd is 14256000
unixYear is 1514764800
ytd is 14270400
unixYear is 1514764800
ytd is 14284800
unixYear is 1514764800
ytd is 14299200
unixYear is 1514764800
ytd is 14313600
unixYear is 1514764800
ytd is 14328000
unixYear is 1514764800
ytd is 14342400
unixYear is 1514764800
ytd is 14356800
unixYear is 1514764800
ytd is 14371200
unixYear is 1514764800
ytd is 14385600
unixYear is 1514764800
yt

32


In [37]:
item_records.iloc[2]['data']

,ts,buyingPrice,buyingCompleted,sellingPrice,sellingCompleted,overallPrice,overallCompleted,date,tsFromCurrent,tsYtd
0,1474603200,188305.0,6.0,181368.0,19.0,183033,25,2016-09-23T00:00:00,59659200.0,22953600.0
1,1474617600,166552.0,28.0,184669.0,14.0,172591,42,2016-09-23T04:00:00,59644800.0,22968000.0
2,1474632000,178507.0,37.0,174035.0,22.0,176840,59,2016-09-23T08:00:00,59630400.0,22982400.0
3,1474646400,177496.0,19.0,166929.0,35.0,170647,54,2016-09-23T12:00:00,59616000.0,22996800.0
4,1474660800,187922.0,6.0,179069.0,32.0,180467,38,2016-09-23T16:00:00,59601600.0,23011200.0
5,1474675200,187868.0,7.0,174465.0,20.0,177940,27,2016-09-23T20:00:00,59587200.0,23025600.0
6,1474689600,193711.0,31.0,183454.0,29.0,188753,60,2016-09-24T00:00:00,59572800.0,23040000.0
7,1474704000,198430.0,2.0,188689.0,24.0,189438,26,2016-09-24T04:00:00,59558400.0,23054400.0
8,1474718400,192284.0,3.0,189487.0,17.0,189907,20,2016-09-24T08:00:00,59544000.0,23068800.0
9,1474732800,174109.0,49.0,189280.0,38.0,180736,87,2016-09-24T12:00:00,59529600.0,23083200.0


In [5]:
timestamps = pd.DataFrame(columns=['ts','tsFromCurrent', 'tsYtd'],data=item_records.iloc[0]['data']['ts'])
lastTs = timestamps['ts'][len(timestamps)-1]

i = 0
print ('Finding timestamps from most recent datapoint.')
while(i<len(timestamps)):
    timestamps['tsFromCurrent'][i] = abs(timestamps['ts'][i] - lastTs)    
    i += 1

years = pd.DataFrame(columns=['year', 'unix'])    
i = 0
for year in range(2015,2050):
    unix = (year - 1970) * 31557600
    years.loc[i] = [year,unix]
    i+=1

i = 0
j = 0
newYear = True
while(i<len(timestamps)):
    unixYear = years.iloc[j]['unix']
    if newYear == True:
        print("Checking entries for year: " + str(years.iloc[j]['year']))
    ytd = abs(timestamps['ts'][i] - unixYear)
    if(ytd < 31557600):
        print("Value found for timestamp: " + str([i]))
        ytd = abs(timestamps.iloc[i]['ts'] - unixYear)
        timestamps['tsYtd'][i] = int(ytd)
        i += 1
        newYear = False
    else: 
        print("Not in this year. Going to next year...")
        j += 1
        newYear = True
            
    
timestamps

Finding timestamps from most recent datapoint.


C:\Users\Jonathan Stav\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Checking entries for year: 2015
Not in this year. Going to next year...
Checking entries for year: 2016
Value found for timestamp: [0]


C:\Users\Jonathan Stav\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Value found for timestamp: [1]
Value found for timestamp: [2]
Value found for timestamp: [3]
Value found for timestamp: [4]
Value found for timestamp: [5]
Value found for timestamp: [6]
Value found for timestamp: [7]
Value found for timestamp: [8]
Value found for timestamp: [9]
Value found for timestamp: [10]
Value found for timestamp: [11]
Value found for timestamp: [12]
Value found for timestamp: [13]
Value found for timestamp: [14]
Value found for timestamp: [15]
Value found for timestamp: [16]
Value found for timestamp: [17]
Value found for timestamp: [18]
Value found for timestamp: [19]
Value found for timestamp: [20]
Value found for timestamp: [21]
Value found for timestamp: [22]
Value found for timestamp: [23]
Value found for timestamp: [24]
Value found for timestamp: [25]
Value found for timestamp: [26]
Value found for timestamp: [27]
Value found for timestamp: [28]
Value found for timestamp: [29]
Value found for timestamp: [30]
Value found for timestamp: [31]
Value found for t

...

Value found for timestamp: [2040]
Value found for timestamp: [2041]
Value found for timestamp: [2042]
Value found for timestamp: [2043]
Value found for timestamp: [2044]
Value found for timestamp: [2045]
Value found for timestamp: [2046]
Value found for timestamp: [2047]
Value found for timestamp: [2048]
Value found for timestamp: [2049]
Value found for timestamp: [2050]
Value found for timestamp: [2051]
Value found for timestamp: [2052]
Value found for timestamp: [2053]
Value found for timestamp: [2054]
Value found for timestamp: [2055]
Value found for timestamp: [2056]
Value found for timestamp: [2057]
Value found for timestamp: [2058]
Value found for timestamp: [2059]
Value found for timestamp: [2060]
Value found for timestamp: [2061]
Value found for timestamp: [2062]
Value found for timestamp: [2063]
Value found for timestamp: [2064]
Value found for timestamp: [2065]
Value found for timestamp: [2066]
Value found for timestamp: [2067]
Value found for timestamp: [2068]
Value found fo

Value found for timestamp: [2369]
Value found for timestamp: [2370]
Value found for timestamp: [2371]
Value found for timestamp: [2372]
Value found for timestamp: [2373]
Value found for timestamp: [2374]
Value found for timestamp: [2375]
Value found for timestamp: [2376]
Value found for timestamp: [2377]
Value found for timestamp: [2378]
Value found for timestamp: [2379]
Value found for timestamp: [2380]
Value found for timestamp: [2381]
Value found for timestamp: [2382]
Value found for timestamp: [2383]
Value found for timestamp: [2384]
Value found for timestamp: [2385]
Value found for timestamp: [2386]
Value found for timestamp: [2387]
Value found for timestamp: [2388]
Value found for timestamp: [2389]
Value found for timestamp: [2390]
Value found for timestamp: [2391]
Value found for timestamp: [2392]
Value found for timestamp: [2393]
Value found for timestamp: [2394]
Value found for timestamp: [2395]
Value found for timestamp: [2396]
Value found for timestamp: [2397]
Value found fo

Value found for timestamp: [2638]
Value found for timestamp: [2639]
Value found for timestamp: [2640]
Value found for timestamp: [2641]
Value found for timestamp: [2642]
Value found for timestamp: [2643]
Value found for timestamp: [2644]
Value found for timestamp: [2645]
Value found for timestamp: [2646]
Value found for timestamp: [2647]
Value found for timestamp: [2648]
Value found for timestamp: [2649]
Value found for timestamp: [2650]
Value found for timestamp: [2651]
Value found for timestamp: [2652]
Value found for timestamp: [2653]
Value found for timestamp: [2654]
Value found for timestamp: [2655]
Value found for timestamp: [2656]
Value found for timestamp: [2657]
Value found for timestamp: [2658]
Value found for timestamp: [2659]
Value found for timestamp: [2660]
Value found for timestamp: [2661]
Value found for timestamp: [2662]
Value found for timestamp: [2663]
Value found for timestamp: [2664]
Value found for timestamp: [2665]
Value found for timestamp: [2666]
Value found fo

Value found for timestamp: [2977]
Value found for timestamp: [2978]
Value found for timestamp: [2979]
Value found for timestamp: [2980]
Value found for timestamp: [2981]
Value found for timestamp: [2982]
Value found for timestamp: [2983]
Value found for timestamp: [2984]
Value found for timestamp: [2985]
Value found for timestamp: [2986]
Value found for timestamp: [2987]
Value found for timestamp: [2988]
Value found for timestamp: [2989]
Value found for timestamp: [2990]
Value found for timestamp: [2991]
Value found for timestamp: [2992]
Value found for timestamp: [2993]
Value found for timestamp: [2994]
Value found for timestamp: [2995]
Value found for timestamp: [2996]
Value found for timestamp: [2997]
Value found for timestamp: [2998]
Value found for timestamp: [2999]
Value found for timestamp: [3000]
Value found for timestamp: [3001]
Value found for timestamp: [3002]
Value found for timestamp: [3003]
Value found for timestamp: [3004]
Value found for timestamp: [3005]
Value found fo

,ts,tsFromCurrent,tsYtd
0,1474603200,59659200,22953600
1,1474617600,59644800,22968000
2,1474632000,59630400,22982400
3,1474646400,59616000,22996800
4,1474660800,59601600,23011200
5,1474675200,59587200,23025600
6,1474689600,59572800,23040000
7,1474704000,59558400,23054400
8,1474718400,59544000,23068800
9,1474732800,59529600,23083200


In [ ]:
i = 0
for year in range(2015,2050):
    unix = (year - 1970) * 31557600
    years.loc[i] = [year,unix]
    i+=1


i = 0
j = 0
while(i<len(timestamps)):
    unixYear = years.iloc[j]['unix']
    print("Checking entries for year: " + str(1970 + int(unixYear/31557600)))
    ytd = abs(timestamps['ts'][i] - unixYear)
    if(ytd < 31557600):
        print("Value found for timestamp: " + str([i]))
        ytd = abs(timestamps.iloc[i]['ts'] - unixYear)
        timestamps['tsYtd'][i] = int(ytd)
        i += 1
    else: 
        print("Not in this year. Going to next year...")
        j += 1

            
print(timestamps)